In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#https://blog.cambridgespark.com/tutorial-preprocessing-text-data-a8969189b779
import pandas as pd
import re
import sklearn
import string
import nltk
import spacy
nltk.download('stopwords')



data = pd.read_excel('/content/gdrive/MyDrive/Praxisprojekt/Trainingdata_train.xlsx', sheet_name = 'sentences')
sentences = data['Sentence'].tolist()
sentences = list((str(s) for s in sentences))
print("Sentences:")
print(sentences)

#muss vom generator object zurück zur liste gemacht werden
sentences = list((s.lower() for s in sentences))
print("Lower-case Sentences:")
print(sentences)

table = str.maketrans('', '', string.punctuation)
sentences = [s.translate(table) for s in sentences]
print("Sentences without punctuation:")
print(sentences)

sentences = [re.sub(r'\d+', 'num', s) for s in sentences]
print("Sentences with 'num' instead of numbers:")
print(sentences)


#remove named entities
nlp = spacy.load("en_core_web_sm")
sentences2 = []
text_no_namedentities = []
for s in sentences:
  document = nlp(s)
  ents = [e.text for e in document.ents]
  for item in document:
    if item.text in ents:
      text_no_namedentities.append('ne')
    else:
        text_no_namedentities.append(item.text)
  sentences2.append(" ".join(text_no_namedentities))
print("Sentences with 'ne' instead of named entities")
print(sentences2)

stopwords = set(nltk.corpus.stopwords.words('english'))
sentences2 = [[word for word in s.split() if word not in stopwords] for s in sentences2]
print("Sentences without stopwords:")
print(sentences2)

stemmer = nltk.stem.PorterStemmer()
sentences2 = [[stemmer.stem(word) for word in s] for s in sentences2]
print("Sentences with stemming:")
print(sentences2)


In [ ]:
#https://blog.cambridgespark.com/tutorial-preprocessing-text-data-a8969189b779
import pandas as pd
import re
import sklearn
import string
import nltk
import spacy
nltk.download('stopwords')


data = pd.read_excel('/content/gdrive/MyDrive/Praxisprojekt/Trainingdata_train.xlsx', sheet_name = 'sentences')
sentences = data['Sentence'].tolist()
sentences = list((str(s) for s in sentences))
print("Sentences:")
print(sentences)

nlp = spacy.load("en_core_web_sm")
sentences2 = []

i = 0
for s in sentences:
  text_no_namedentities = []
  i += 1
  if i == 5:
    break
  document = nlp(s)
  ents = [e.text for e in document.ents]  #enthält alle named entities dieses satzes
  print("Satz ",i," named entities:")
  print(ents)
  for item in document:
    print(item.text)
    if item.text in ents:
      text_no_namedentities.append('ne')
    else:
        text_no_namedentities.append(item.text)
  sentences2.append(" ".join(text_no_namedentities))
print("Sentences with 'ne' instead of named entities")
print(sentences2)

In [12]:
import pandas as pd
import re
import sklearn
import string
import nltk
import spacy
nltk.download('stopwords')


data = pd.read_excel('/content/gdrive/MyDrive/Praxisprojekt/Trainingdata_train.xlsx', sheet_name = 'sentences')
sentences = data['Sentence'].tolist()
sentences = list((str(s) for s in sentences))
print("Sentences:")
print(sentences)

nlp = spacy.load("en_core_web_sm")
sentences2 = []

for s in sentences:
  text_no_namedentities = []
  document = nlp(s)
  
  for item in document:
    if item.ent_type: #falls es ein name ist
      #text_no_namedentities.append('ne')        # durch ne ersetzen
      pass                                        # oder ganz auslassen
    else:
        text_no_namedentities.append(item.text)
  sentences2.append(" ".join(text_no_namedentities))
print("Sentences with 'ne' instead of named entities")
print(sentences2)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Sentences:
['miley and liam fighting false rumors swirl that theyre in a feud over a supposed prenup', 'are miley cyrus and liam hemsworth fighting', 'a new report claims that cyrus and hemsworth had a huge fight over their supposed prenup while they were making their wedding plans', 'according to the report cyrus 24 and hemsworth 27 have been planning their wedding in the himalayan mountains this summer', 'but there has reportedly been tension between them over having a prenup', 'cyrus reportedly told hemsworth she wanted him to sign a prenup before they got married so she could protect her estimated 200 million net worth', 'and hemsworth was reportedly shocked and annoyed with miley and accused her of not trusting him', 'are miley and liam getting married this summer click here for details', 'sources say miley cyrus and liam hemsworth are fighting over a prenup photo

In [16]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import re
import sklearn
import string
import nltk
nltk.download('stopwords')
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score



class PreprocessorTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
      pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, data, y=None):
        sentences = data['Sentence'].tolist()
        sentences = list((str(s) for s in sentences))

        # Alles klein schreiben
        # muss vom generator object zurück zur Liste gemacht werden
        sentences = list((s.lower() for s in sentences))

        # Satzzeichen entfernen
        table = str.maketrans('', '', string.punctuation)
        sentences = [s.translate(table) for s in sentences]

        # ersetzt Zahlen durch den String "num"
        #sentences = [re.sub(r'\d+', 'num', s) for s in sentences]

        #remove named entities
        nlp = spacy.load("en_core_web_sm")
        sentences2 = []
        for s in sentences:
          text_no_namedentities = []
          document = nlp(s)
          for item in document:
            if item.ent_type: #falls es ein name ist
              #text_no_namedentities.append('ne')        # durch ne ersetzen
              pass                                        # oder ganz auslassen
            else:
              text_no_namedentities.append(item.text)
          sentences2.append(" ".join(text_no_namedentities))

        # entfernt stopwords
        stopwords = set(nltk.corpus.stopwords.words('english')+ ['liam', 'hemsworth', 'cyrus'])
        sentences2 = [[word for word in s.split() if word not in stopwords] for s in sentences2]
        
        # führte Wort-Stemming durch
        stemmer = nltk.stem.PorterStemmer()
        sentences2 = [[stemmer.stem(word) for word in s] for s in sentences2]

        return sentences2
    



class BagOfWordsForEachSentenceTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
      pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, sentences, y=None):
        print('transformed called')
        dics = []

        #für jeden Satz wird ein Dictionary konstruiert
        for s in sentences:
          c = Counter(s)
          dic = []
          for key in c:
            dic[key]=(c[key])
          dics.append(dic)

        return (sentences, dics)
    
    
class BagOfWordsTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        # Kann man eventuell besser machen und muss man nicht so machen
        self.bigdict = {}
        self.training = True
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, sentences, y=None):
        if self.training:
          from collections import Counter
          self.bigdict = {}
          for s in sentences:
            c = Counter(s)
            for key in c:
              if key in self.bigdict:
                self.bigdict[key] = self.bigdict[key] + c[key]
              else:
                self.bigdict[key] = 1
          self.training = False

        return (sentences, self.bigdict)


class SentenceToVectorTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
      pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, sentence_dict_tuple, y=None):
        # Tupel zerlegen
        sentence_fragments, dicts = sentence_dict_tuple
        sorted_keys = [key for key in dicts.keys()]
        retval = []
        for sentence_fragment in sentence_fragments:
          vec = []
          for key in sorted_keys:
            if key in sentence_fragment:
              vec.append(1)
            else:
              vec.append(0)
          retval.append(vec)
        return retval


# Erstelle Pipeline
pipe_LR = sklearn.pipeline.make_pipeline(PreprocessorTransformer(), BagOfWordsTransformer(), SentenceToVectorTransformer(), sklearn.linear_model.LogisticRegression())
pipe_NB = sklearn.pipeline.make_pipeline(PreprocessorTransformer(), BagOfWordsTransformer(), SentenceToVectorTransformer(), GaussianNB())

# Lade Datensätze
data_train = pd.read_excel('/content/gdrive/MyDrive/Praxisprojekt/Trainingdata_train.xlsx', sheet_name = 'sentences')
data_train.drop(['SUBJindl', 'SUBJsrce', 'SUBJrhet', 'SUBJster', 'SUBJspee', 'SUBJinspe', 'SUBJprop', 'SUBJpolit'], axis=1, inplace=True)

data_test = pd.read_excel('/content/gdrive/MyDrive/Praxisprojekt/Trainingdata_test.xlsx', sheet_name = 'sentences')
data_test.drop(['SUBJindl', 'SUBJsrce', 'SUBJrhet', 'SUBJster', 'SUBJspee', 'SUBJinspe', 'SUBJprop', 'SUBJpolit'], axis=1, inplace=True)

#y_pred_LR = pipe_LR.predict(data_test)
#y_pred_NB = pipe_NB.predict(data_test)

#print("results for sentiment:")
#y_lang = data_train.SUBJlang.to_numpy().astype(int)
#pipe_LR.fit(data_train, y_lang)
#y_lang_test = data_test.SUBJlang.to_numpy()
#y_pred_LR = pipe_LR.predict(data_test)
#accuracy_LR = accuracy_score(y_pred_LR, y_lang_test)
#print(f'Accuracy logistic Regression: {accuracy_LR}')
#pipe_NB.fit(data_train, y_lang)
#y_pred_NB = pipe_NB.predict(data_test)
#accuracy_NB = accuracy_score(y_pred_NB, y_lang_test)
#print(f'Accuracy Navie Bayes: {accuracy_NB}')

#pipe_LR = sklearn.pipeline.make_pipeline(PreprocessorTransformer(), BagOfWordsTransformer(), SentenceToVectorTransformer(), sklearn.linear_model.LogisticRegression())
#pipe_NB = sklearn.pipeline.make_pipeline(PreprocessorTransformer(), BagOfWordsTransformer(), SentenceToVectorTransformer(), GaussianNB())

print("results for opinion:")
y_opin = data_train.SUBJopin.to_numpy().astype(int)
pipe_LR.fit(data_train, y_opin)
y_opin_test = data_test.SUBJopin.to_numpy()
accuracy_LR = accuracy_score(y_pred_LR, y_opin_test)
print(f'Accuracy logistic Regression: {accuracy_LR}')
pipe_NB.fit(data_train, y_opin)
accuracy_NB = accuracy_score(y_pred_NB, y_opin_test)
print(f'Accuracy Navie Bayes: {accuracy_NB}')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
results for opinion:
Accuracy logistic Regression: 0.16796875
Accuracy Navie Bayes: 0.2734375


In [19]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import re
import sklearn
import string
import nltk
nltk.download('stopwords')
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score



class PreprocessorTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
      pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, data, y=None):
        sentences = data['Sentence'].tolist()
        sentences = list((str(s) for s in sentences))

        # Alles klein schreiben
        # muss vom generator object zurück zur Liste gemacht werden
        sentences = list((s.lower() for s in sentences))

        # Satzzeichen entfernen
        table = str.maketrans('', '', string.punctuation)
        sentences = [s.translate(table) for s in sentences]

        # ersetzt Zahlen durch den String "num"
        sentences = [re.sub(r'\d+', 'num', s) for s in sentences]

        
        # entfernt stopwords
        stopwords = set(nltk.corpus.stopwords.words('english')+ ['liam', 'hemsworth', 'cyrus'])
        sentences = [[word for word in s.split() if word not in stopwords] for s in sentences]
        
        # führte Wort-Stemming durch
        stemmer = nltk.stem.PorterStemmer()
        sentences = [[stemmer.stem(word) for word in s] for s in sentences]

        return sentences
    



class BagOfWordsForEachSentenceTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
      pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, sentences, y=None):
        print('transformed called')
        dics = []

        #für jeden Satz wird ein Dictionary konstruiert
        for s in sentences:
          c = Counter(s)
          dic = []
          for key in c:
            dic[key]=(c[key])
          dics.append(dic)

        return (sentences, dics)
    
    
class BagOfWordsTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        # Kann man eventuell besser machen und muss man nicht so machen
        self.bigdict = {}
        self.training = True
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, sentences, y=None):
        if self.training:
          from collections import Counter
          self.bigdict = {}
          for s in sentences:
            c = Counter(s)
            for key in c:
              if key in self.bigdict:
                self.bigdict[key] = self.bigdict[key] + c[key]
              else:
                self.bigdict[key] = 1
          self.training = False

        return (sentences, self.bigdict)


class SentenceToVectorTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
      pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, sentence_dict_tuple, y=None):
        # Tupel zerlegen
        sentence_fragments, dicts = sentence_dict_tuple
        sorted_keys = [key for key in dicts.keys()]
        retval = []
        for sentence_fragment in sentence_fragments:
          vec = []
          for key in sorted_keys:
            if key in sentence_fragment:
              vec.append(1)
            else:
              vec.append(0)
          retval.append(vec)
        return retval


# Erstelle Pipeline
pipe_LR = sklearn.pipeline.make_pipeline(PreprocessorTransformer(), BagOfWordsTransformer(), SentenceToVectorTransformer(), sklearn.linear_model.LogisticRegression())
pipe_NB = sklearn.pipeline.make_pipeline(PreprocessorTransformer(), BagOfWordsTransformer(), SentenceToVectorTransformer(), GaussianNB())

# Lade Datensätze
data_train = pd.read_excel('/content/gdrive/MyDrive/Praxisprojekt/Trainingdata_train.xlsx', sheet_name = 'sentences')
data_train.drop(['SUBJindl', 'SUBJsrce', 'SUBJrhet', 'SUBJster', 'SUBJspee', 'SUBJinspe', 'SUBJprop', 'SUBJpolit'], axis=1, inplace=True)

data_test = pd.read_excel('/content/gdrive/MyDrive/Praxisprojekt/Trainingdata_test.xlsx', sheet_name = 'sentences')
data_test.drop(['SUBJindl', 'SUBJsrce', 'SUBJrhet', 'SUBJster', 'SUBJspee', 'SUBJinspe', 'SUBJprop', 'SUBJpolit'], axis=1, inplace=True)


print("results for sentiment:")
y_lang = data_train.SUBJlang.to_numpy().astype(int)
pipe_LR.fit(data_train, y_lang)
y_lang_test = data_test.SUBJlang.to_numpy()
y_pred_LR = pipe_LR.predict(data_test)
accuracy_LR = accuracy_score(y_pred_LR, y_lang_test)
print(f'Accuracy logistic Regression: {accuracy_LR}')
pipe_NB.fit(data_train, y_lang)
y_pred_NB = pipe_NB.predict(data_test)
accuracy_NB = accuracy_score(y_pred_NB, y_lang_test)
print(f'Accuracy Naive Bayes: {accuracy_NB}')


print("results for opinion:")
y_opin = data_train.SUBJopin.to_numpy().astype(int)
pipe_LR.fit(data_train, y_opin)
y_opin_test = data_test.SUBJopin.to_numpy()
y_pred_LR = pipe_LR.predict(data_test)
accuracy_LR = accuracy_score(y_pred_LR, y_opin_test)
print(f'Accuracy logistic Regression: {accuracy_LR}')
pipe_NB.fit(data_train, y_opin)
y_pred_NB = pipe_NB.predict(data_test)
accuracy_NB = accuracy_score(y_pred_NB, y_opin_test)
print(f'Accuracy Naive Bayes: {accuracy_NB}')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
results for sentiment:
Accuracy logistic Regression: 0.529296875
Accuracy Navie Bayes: 0.501953125
results for opinion:
Accuracy logistic Regression: 0.646484375
Accuracy Navie Bayes: 0.591796875
